In [10]:
def maximize_coupons(item_prices, coupons):
    """
    Splits items into batches to maximize the total benefit from coupons with different thresholds and benefits.

    Parameters:
    - item_prices: List of item prices (list of floats or ints).
    - coupons: List of coupon dictionaries, each with 'threshold' and 'benefit'.

    Returns:
    - batches: List of batches, where each batch is a dictionary with keys:
        - 'items': List of item prices in the batch.
        - 'total': Total price of the batch.
        - 'coupon_threshold': The threshold of the coupon applied to the batch (if any).
        - 'coupon_benefit': The benefit of the coupon applied to the batch (if any).
    """
    from itertools import combinations
    N = len(item_prices)
    item_indices = list(range(N))
    item_prices = [float(price) for price in item_prices]

    # Generate all possible subsets of items
    subsets = []
    for r in range(1, N+1):
        for indices in combinations(item_indices, r):
            subset_mask = 0
            total_price = 0
            for idx in indices:
                subset_mask |= 1 << idx
                total_price += item_prices[idx]
            # Check for each coupon
            for coupon in coupons:
                threshold = coupon['threshold']
                benefit = coupon['benefit']
                if total_price >= threshold:
                    subsets.append({
                        'mask': subset_mask,
                        'items': indices,
                        'total_price': total_price,
                        'coupon_threshold': threshold,
                        'coupon_benefit': benefit
                    })
                    # Break to only consider the most beneficial coupon for this subset
                    break  # Remove this if multiple coupons per batch are allowed

    # Dynamic Programming to maximize total coupon benefit
    max_mask = (1 << N) - 1
    dp = [(-1, None)] * (max_mask + 1)  # (total_benefit, previous_state)
    dp[0] = (0, None)

    for state in range(max_mask + 1):
        if dp[state][0] == -1:
            continue
        for subset in subsets:
            subset_mask = subset['mask']
            if state & subset_mask == 0:
                new_state = state | subset_mask
                new_benefit = dp[state][0] + subset['coupon_benefit']
                if dp[new_state][0] < new_benefit:
                    dp[new_state] = (new_benefit, (state, subset))

    # Reconstruct batches from DP table
    batches = []
    state = max_mask
    while state != 0:
        if dp[state][1] is None:
            # Remaining items that couldn't be used with any coupon
            remaining_items = [idx for idx in range(N) if (state & (1 << idx))]
            batch_items = [item_prices[idx] for idx in remaining_items]
            batches.append({
                'items': batch_items,
                'total': sum(batch_items),
                'coupon_threshold': None,
                'coupon_benefit': 0
            })
            break
        prev_state, subset = dp[state][1]
        batch_items = [item_prices[idx] for idx in subset['items']]
        batches.append({
            'items': batch_items,
            'total': subset['total_price'],
            'coupon_threshold': subset['coupon_threshold'],
            'coupon_benefit': subset['coupon_benefit']
        })
        state = prev_state

    batches.reverse()  # Reverse to get batches in the original order
    return batches

# Usage:

item_prices = [6.64, 6.57, 94.90, 7.21, 189.24, 4.64, 30.5, 9.68, 29.35, 2.04, 3.50]
coupons = [
    {'threshold': 269, 'benefit': 40},
    {'threshold': 169, 'benefit': 25},
    {'threshold': 89, 'benefit': 12},
    {'threshold': 39, 'benefit': 5}
]

batches = maximize_coupons(item_prices, coupons)

print("Batches:")
total_benefit = 0
for i, batch in enumerate(batches, 1):
    if batch['coupon_threshold']:
        coupon_info = f"Coupon Applied: Threshold = {batch['coupon_threshold']}, Benefit = {batch['coupon_benefit']}"
    else:
        coupon_info = "No Coupon Applied"
    total_benefit += batch['coupon_benefit']
    print(f"Batch {i}: Items = {batch['items']}, Total = {batch['total']:.2f}, {coupon_info}")
print(f"Total Benefit: {total_benefit}")

Batches:
Batch 1: Items = [94.9], Total = 94.90, Coupon Applied: Threshold = 89, Benefit = 12
Batch 2: Items = [6.64, 6.57, 7.21, 189.24, 4.64, 30.5, 9.68, 29.35, 2.04, 3.5], Total = 289.37, Coupon Applied: Threshold = 269, Benefit = 40
Total Benefit: 52
